Import library 

In [23]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.10f}'.format

import pickle
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
import tensorflow as tf
import seaborn as sns
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8

from pylab import rcParams

from sklearn.preprocessing import Normalizer, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold

import os

RANDOM_SEED = 42

# GPU 0번
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [24]:
df = pd.read_csv("ano/train.csv")
df.head(n=5)

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,3,-1.3583540616,-1.3401630747,1.7732093426,0.3797795930,-0.5031981333,1.8004993808,0.7914609565,0.2476757866,-1.5146543226,...,0.2479981535,0.7716794019,0.9094122623,-0.6892809565,-0.3276418337,-0.1390965715,-0.0553527940,-0.0597518406,4.9837210927,-0.9949717454
1,4,-0.9662717116,-0.1852260081,1.7929933396,-0.8632912750,-0.0103088796,1.2472031675,0.2376089398,0.3774358747,-1.3870240627,...,-0.1083004520,0.0052735968,-0.1903205187,-1.1755753319,0.6473760346,-0.2219288445,0.0627228487,0.0614576285,1.4182910641,-0.9949717454
2,6,-0.4259658844,0.9605230449,1.1411093423,-0.1682520798,0.4209868808,-0.0297275517,0.4762009487,0.2603143331,-0.5686713757,...,-0.2082535147,-0.5598247963,-0.0263976680,-0.3714265832,-0.2327938167,0.1059147791,0.2538442247,0.0810802569,-0.2561307902,-0.9949599972
3,8,-0.6442694423,1.4179635455,1.0743803764,-0.4921990185,0.9489340948,0.4281184628,1.1206313584,-3.8078642387,0.6153747307,...,1.9434653398,-1.0154547098,0.0575035299,-0.6497090056,-0.4152665662,-0.0516342969,-1.2069210809,-1.0853391883,0.2626982463,-0.9949012559
4,9,-0.8942860822,0.2861571963,-0.1131922127,-0.2715261301,2.6695986596,3.7218180611,0.3701451277,0.8510844432,-0.3920475868,...,-0.0734251001,-0.2680916322,-0.2042326699,1.0115918019,0.3732046801,-0.3841573077,0.0117473565,0.1424043299,0.9948997415,-0.9949012559


In [25]:
df.describe()

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
count,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,...,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000,113842.0000000000
mean,142025.5278368265,0.0001972910,0.0012887948,0.0097173629,-0.0041692083,0.0004749962,0.0051411577,0.0057690239,-0.0024505883,-0.0021070687,...,-0.0012424691,0.0000004088,-0.0013174176,-0.0008841494,0.0016801536,-0.0002933701,-0.0002337128,-0.0005080546,0.9272530175,0.1162315431
std,82248.5453921828,1.9510604522,1.6510635788,1.4969157271,1.4126331716,1.3675334867,1.3305827734,1.2041108133,1.1855042538,1.0954151622,...,0.7220012926,0.7238290619,0.6360608349,0.6058543652,0.5200694420,0.4809785789,0.3995054883,0.3561296472,3.4129328933,0.5581608630
min,3.0000000000,-56.4075096313,-72.7157275629,-32.4541981863,-5.6006071412,-42.1478983728,-26.1605059358,-41.5067960833,-50.9433688677,-13.4340663182,...,-22.7575398591,-8.8870171409,-44.8077352038,-2.8248489029,-10.2953970750,-1.8553553378,-9.8952440476,-9.6179154524,-0.3074128415,-0.9949717454
25%,70796.7500000000,-0.9234789639,-0.5956024974,-0.8838765993,-0.8537282566,-0.6898532168,-0.7660942317,-0.5520711262,-0.2094919796,-0.6474770197,...,-0.2297098048,-0.5402664835,-0.1621803300,-0.3555819797,-0.3154703270,-0.3261599344,-0.0708471352,-0.0532485515,-0.2305596311,-0.3603043974
50%,141722.0000000000,0.0120736214,0.0663902608,0.1838678153,-0.0193594879,-0.0540596089,-0.2724355892,0.0390361877,0.0209696520,-0.0521569296,...,-0.0302814776,0.0083458070,-0.0122609847,0.0405734902,0.0182780212,-0.0528153757,0.0015018849,0.0111582113,-0.0006986655,-0.0025904910
75%,213359.5000000000,1.3153733393,0.8016868438,1.0371202905,0.7422077847,0.6142135083,0.4052848744,0.5687501336,0.3283030470,0.5907047932,...,0.1860012866,0.5287508178,0.1474742894,0.4382245391,0.3539892820,0.2408375726,0.0912790742,0.0778514865,0.7685321037,0.6406530857
max,284803.0000000000,2.4549299912,21.4672029943,4.1878105990,16.4912171737,34.8016658767,23.9178371266,44.0544613632,20.0072083651,10.3928888247,...,27.2028391573,8.3619851917,22.5284116898,4.0228658904,7.5195886787,3.1192945290,11.1357398446,33.8478078189,180.1010270384,1.0349510685


In [26]:
df_val = pd.read_csv("ano/val.csv")
df_val.head(n=5)

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
0,10,-0.3382617524,1.1195933764,1.0443665516,-0.2221872767,0.4993608065,-0.2467611006,0.6515832065,0.0695385865,-0.7367273164,...,-0.6337526424,-0.1207940841,-0.3850499253,-0.0697330460,0.0941988340,0.2462193046,0.0830756493,-0.2559910571,-0.9948777594,0
1,22,0.9624960699,0.3284610261,-0.1714790542,2.1092040677,1.1295655713,1.6960376857,0.1077116073,0.5215021638,-1.1913111021,...,0.4024916614,-0.0485082212,-1.3718662945,0.3908138854,0.1999636575,0.0163706433,-0.0146053277,0.1689373297,-0.9947837733,0
2,63,1.1455243873,0.5750679681,0.1940084600,2.5981917450,-0.0922100227,-1.0444295837,0.5315875930,-0.2418881266,-0.8962871739,...,-0.1197031003,-0.0765095276,0.6913204314,0.6339839771,0.0487412745,-0.0531919416,0.0162514988,0.1694962621,-0.9945018151,0
3,69,0.9270602489,-0.3236839717,0.3875848310,0.5444739886,0.2467873947,1.6503578045,-0.4275760175,0.6153706984,0.2262782750,...,0.0793594012,0.0966317457,-0.9925686689,0.0850958289,0.3774472053,0.0360964213,-0.0059602861,0.3313072032,-0.9944665703,0
4,83,-3.0052367224,2.6001382022,1.4836907373,-2.4184729551,0.3063264066,-0.8245745319,2.0654256393,-1.8293465894,4.0092585167,...,-0.1812684838,-0.1637465041,0.5158205258,0.1363184320,0.4600539675,-0.2512586378,-1.1057511571,-0.2870118074,-0.9943725843,0


In [27]:
df_test = pd.read_csv("ano/test.csv")
df_test.head(n=5)

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,AAAA0x1,-1.3598071337,-0.0727811733,2.5363467380,1.3781552243,-0.3383207699,0.4623877778,0.2395985541,0.0986979013,0.3637869696,...,-0.0183067779,0.2778375756,-0.1104739102,0.0669280749,0.1285393583,-0.1891148439,0.1335583767,-0.0210530535,1.7832739468,-0.9949834937
1,AAAA0x2,1.1918571113,0.2661507121,0.1664801134,0.4481540785,0.0600176493,-0.0823608088,-0.0788029833,0.0851016549,-0.2554251281,...,-0.2257752480,-0.6386719528,0.1012880213,-0.3398464755,0.1671704044,0.1258945324,-0.0089830991,0.0147241692,-0.2698246349,-0.9949834937
2,AAAA0x5,-1.1582330935,0.8777367548,1.5487178465,0.4030339340,-0.4071933773,0.0959214625,0.5929407454,-0.2705326772,0.8177393082,...,-0.0094306971,0.7982784946,-0.1374580796,0.1412669838,-0.2060095876,0.5022922242,0.2194222295,0.2151531475,0.6705791937,-0.9949599972
3,AAAA0x7,1.2296576345,0.1410035070,0.0453707736,1.2026127367,0.1918809886,0.2727081229,-0.0051590029,0.0812129399,0.4649599948,...,-0.1677162658,-0.2707097262,-0.1541037868,-0.7800554150,0.7501369358,-0.2572368459,0.0345074297,0.0051677689,-0.2376860197,-0.9949365007
4,AAAA0xc,0.3849782152,0.6161094592,-0.8742997026,-0.0940186260,2.9245843784,3.3170271683,0.4704546718,0.5382472284,-0.5588946124,...,0.0499236859,0.2384215122,0.0091298686,0.9967102096,-0.7673148272,-0.4922082953,0.0424724419,-0.0543373884,-0.1678194648,-0.9948660111


In [28]:
print('Check number of Rows and Columns')
print(df.shape)

print('Check if any Missing values')
df.isnull().values.any()

Check number of Rows and Columns
(113842, 31)
Check if any Missing values


False

In [29]:
data = df.drop(['ID'], axis=1)

data_val_y = df_val['Class']
data_val = df_val.drop(['ID','Class'], axis=1)

data_test = df_test.drop(['ID'], axis=1)

In [30]:
X_train = data.values
X_test = data_test.values
X_val = data_val.values

In [31]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(113842, 30)
(142503, 30)
(28462, 30)


데이터 정규화

In [32]:
pipeline = Pipeline([('normalizer', Normalizer()),
                     ('scaler', MinMaxScaler())])

X_train = pipeline.fit_transform(X_train)
X_val = pipeline.transform(X_val)
X_test = pipeline.transform(X_test)

Autoencoder - 2_layer encoder, 2_layer decoder

In [33]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

nb_epoch = 100
batch_size = 512
adamax = keras.optimizers.Adam(learning_rate=0.001)

    
input_dim = X_train.shape[1]
encoding_dim = 150
input_layer = Input(shape=(input_dim, ))

encoder = Dense(encoding_dim, activation="tanh")(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
decoder = Dense(input_dim)(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer=adamax, 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath=f"model.h5",
                               verbose=0,
                               save_best_only=True)

tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)

history = autoencoder.fit(X_train, X_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_val, X_val),
                    verbose=1,
                    callbacks=[checkpointer, tensorboard]).history

Train on 113842 samples, validate on 28462 samples
Epoch 1/100
 51712/113842 [============>.................] - ETA: 0s - loss: 0.0309 - accuracy: 0.1253

2022-09-28 13:50:34.400127: I tensorflow/core/profiler/lib/profiler_session.cc:225] Profiler session started.
2022-09-28 13:50:34.400160: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1307] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:50:34.400167: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1346] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:50:34.404051: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1329] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:50:34.404078: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:88]  GpuTracer has collect

113842/113842 [==============================] - 1s 5us/sample - loss: 0.0177 - accuracy: 0.2887 - val_loss: 0.0038 - val_accuracy: 0.5605
Epoch 2/100
113842/113842 [==============================] - 0s 4us/sample - loss: 0.0021 - accuracy: 0.7030 - val_loss: 0.0010 - val_accuracy: 0.8019
Epoch 3/100
113842/113842 [==============================] - 1s 9us/sample - loss: 6.2837e-04 - accuracy: 0.8509 - val_loss: 3.6639e-04 - val_accuracy: 0.8856
Epoch 4/100
113842/113842 [==============================] - 1s 6us/sample - loss: 2.7098e-04 - accuracy: 0.9024 - val_loss: 1.9108e-04 - val_accuracy: 0.9213
Epoch 5/100
113842/113842 [==============================] - 1s 5us/sample - loss: 1.4760e-04 - accuracy: 0.9315 - val_loss: 1.1045e-04 - val_accuracy: 0.9418
Epoch 6/100
113842/113842 [==============================] - 1s 6us/sample - loss: 8.8929e-05 - accuracy: 0.9470 - val_loss: 7.1017e-05 - val_accuracy: 0.9510
Epoch 7/100
113842/113842 [==============================] - 1s 8us/sample

113842/113842 [==============================] - 1s 4us/sample - loss: 9.8473e-06 - accuracy: 0.9810 - val_loss: 5.3514e-06 - val_accuracy: 0.9838
Epoch 53/100
113842/113842 [==============================] - 1s 12us/sample - loss: 9.0788e-06 - accuracy: 0.9817 - val_loss: 9.3448e-06 - val_accuracy: 0.9826
Epoch 54/100
113842/113842 [==============================] - 1s 7us/sample - loss: 9.0910e-06 - accuracy: 0.9817 - val_loss: 6.7775e-06 - val_accuracy: 0.9842
Epoch 55/100
113842/113842 [==============================] - 1s 6us/sample - loss: 1.0131e-05 - accuracy: 0.9805 - val_loss: 7.8223e-06 - val_accuracy: 0.9838
Epoch 56/100
113842/113842 [==============================] - 1s 7us/sample - loss: 8.5664e-06 - accuracy: 0.9826 - val_loss: 9.4631e-06 - val_accuracy: 0.9794
Epoch 57/100
113842/113842 [==============================] - 1s 8us/sample - loss: 9.1416e-06 - accuracy: 0.9808 - val_loss: 1.0382e-05 - val_accuracy: 0.9781
Epoch 58/100
113842/113842 [========================

원래값 - 예측값 ** 2으로 MSE 측정

정상 데이터는 원래값에 근사할것

비정상 데이터는 원래값과 차이가 있을것

mse가 임계값 이상이면 비정상 데이터라고 판단

1. 임계값 
평균 + 2 * 표준편차 -> 더 좋은 성능을 보임

2. 임계값
중앙값 + 1.5 * IQR(사분위수 Q3 - Q1)

In [34]:
autoencoder = load_model(f'model.h5')
predictions = autoencoder.predict(X_val)
mse = np.mean(np.power(X_val - predictions, 2), axis=1)
    
error_df = pd.DataFrame({'reconstruction_error': mse, 'true_class' : data_val_y})
error_df.describe()

means = error_df.describe()['reconstruction_error'].loc['mean']
stds = error_df.describe()['reconstruction_error'].loc['std']

median = error_df.describe()['reconstruction_error'].loc['50%']
IQR = error_df.describe()['reconstruction_error'].loc['75%'] - error_df.describe()['reconstruction_error'].loc['25%']

threshold = means + 2 * stds
threshold2 = median + 1.5 * IQR


tests = np.where(mse>=threshold, 1, 0)

df = pd.DataFrame({'mse' : mse, 'test' : tests, 'true' : data_val_y})
df[df['test'] == 1]['true'].value_counts()

0    158
1      9
Name: true, dtype: int64

kfold로 trainset을 일정량씩 사용하여 ensemble

In [35]:
kfold_len = 5
kfold = KFold(n_splits=kfold_len)

nb_epoch = 100
batch_size = 512
adamax = keras.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)


for idx, (train_index, test_index) in enumerate(kfold.split(X_train)):  # feautres 데이터를 위에서 지정한 kfold 숫자로 분할
    print('idx : ', idx)
    x_train = X_train[train_index]
    x_test = X_train[test_index]
    
    input_dim = X_train.shape[1]
    encoding_dim = 100
    input_layer = Input(shape=(input_dim, ))
    
    encoder = Dense(encoding_dim, activation="tanh")(input_layer)
    encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
    decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
    decoder = Dense(input_dim)(decoder)
    
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    autoencoder.compile(optimizer=adamax, 
                        loss=root_mean_squared_error, 
                        metrics=['accuracy'])
    
    checkpointer = ModelCheckpoint(filepath=f"model{idx}.h5",
                                   verbose=0,
                                   save_best_only=True)
    
    tensorboard = TensorBoard(log_dir='./logs',
                              histogram_freq=0,
                              write_graph=True,
                              write_images=True)
    
    history = autoencoder.fit(x_train, x_train,
                        epochs=nb_epoch,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(x_test, x_test),
                        verbose=1,
                        callbacks=[checkpointer, tensorboard]).history

idx :  0
Train on 91073 samples, validate on 22769 samples
Epoch 1/100
11776/91073 [==>...........................] - ETA: 3s - loss: 0.2468 - accuracy: 0.0296

2022-09-28 13:51:49.219920: I tensorflow/core/profiler/lib/profiler_session.cc:225] Profiler session started.
2022-09-28 13:51:49.220030: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1307] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:51:49.220062: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1346] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:51:49.233905: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1329] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:51:49.233994: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:88]  GpuTracer has collect

91073/91073 [==============================] - 1s 9us/sample - loss: 0.1275 - accuracy: 0.2580 - val_loss: 0.0949 - val_accuracy: 0.3156
Epoch 2/100
91073/91073 [==============================] - 0s 5us/sample - loss: 0.0734 - accuracy: 0.5186 - val_loss: 0.0694 - val_accuracy: 0.5323
Epoch 3/100
91073/91073 [==============================] - 1s 7us/sample - loss: 0.0523 - accuracy: 0.6743 - val_loss: 0.0477 - val_accuracy: 0.6986
Epoch 4/100
91073/91073 [==============================] - 1s 7us/sample - loss: 0.0380 - accuracy: 0.7815 - val_loss: 0.0358 - val_accuracy: 0.7813
Epoch 5/100
91073/91073 [==============================] - 1s 10us/sample - loss: 0.0297 - accuracy: 0.8210 - val_loss: 0.0295 - val_accuracy: 0.8138
Epoch 6/100
91073/91073 [==============================] - 1s 7us/sample - loss: 0.0260 - accuracy: 0.8383 - val_loss: 0.0268 - val_accuracy: 0.8198
Epoch 7/100
91073/91073 [==============================] - 1s 7us/sample - loss: 0.0224 - accuracy: 0.8546 - val_loss

91073/91073 [==============================] - 1s 8us/sample - loss: 0.0047 - accuracy: 0.9734 - val_loss: 0.0049 - val_accuracy: 0.9723
Epoch 56/100
91073/91073 [==============================] - 1s 10us/sample - loss: 0.0047 - accuracy: 0.9728 - val_loss: 0.0050 - val_accuracy: 0.9646
Epoch 57/100
91073/91073 [==============================] - 1s 9us/sample - loss: 0.0046 - accuracy: 0.9726 - val_loss: 0.0044 - val_accuracy: 0.9733
Epoch 58/100
91073/91073 [==============================] - 1s 10us/sample - loss: 0.0046 - accuracy: 0.9729 - val_loss: 0.0048 - val_accuracy: 0.9664
Epoch 59/100
91073/91073 [==============================] - 1s 8us/sample - loss: 0.0047 - accuracy: 0.9722 - val_loss: 0.0043 - val_accuracy: 0.9733
Epoch 60/100
91073/91073 [==============================] - 0s 4us/sample - loss: 0.0045 - accuracy: 0.9738 - val_loss: 0.0041 - val_accuracy: 0.9746
Epoch 61/100
91073/91073 [==============================] - 1s 12us/sample - loss: 0.0040 - accuracy: 0.9745 - 

2022-09-28 13:53:03.645882: I tensorflow/core/profiler/lib/profiler_session.cc:225] Profiler session started.
2022-09-28 13:53:03.645915: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1307] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:53:03.645921: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1346] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:53:03.649426: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1329] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:53:03.649447: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:88]  GpuTracer has collect

...

91074/91074 [==============================] - 1s 8us/sample - loss: 0.1528 - accuracy: 0.1774 - val_loss: 0.0895 - val_accuracy: 0.4754
Epoch 2/100
91074/91074 [==============================] - 1s 7us/sample - loss: 0.0722 - accuracy: 0.4991 - val_loss: 0.0555 - val_accuracy: 0.6754
Epoch 3/100
91074/91074 [==============================] - 1s 7us/sample - loss: 0.0463 - accuracy: 0.7062 - val_loss: 0.0397 - val_accuracy: 0.7663
Epoch 4/100
91074/91074 [==============================] - 1s 7us/sample - loss: 0.0339 - accuracy: 0.7867 - val_loss: 0.0308 - val_accuracy: 0.8223
Epoch 5/100
91074/91074 [==============================] - 1s 7us/sample - loss: 0.0268 - accuracy: 0.8413 - val_loss: 0.0252 - val_accuracy: 0.8638
Epoch 6/100
91074/91074 [==============================] - 1s 9us/sample - loss: 0.0220 - accuracy: 0.8696 - val_loss: 0.0216 - val_accuracy: 0.8771
Epoch 7/100
91074/91074 [==============================] - 1s 6us/sample - loss: 0.0195 - accuracy: 0.8879 - val_loss:

Epoch 56/100
91074/91074 [==============================] - 1s 6us/sample - loss: 0.0091 - accuracy: 0.9625 - val_loss: 0.0096 - val_accuracy: 0.9584
Epoch 57/100
91074/91074 [==============================] - 1s 7us/sample - loss: 0.0092 - accuracy: 0.9619 - val_loss: 0.0092 - val_accuracy: 0.9617
Epoch 58/100
91074/91074 [==============================] - 1s 7us/sample - loss: 0.0092 - accuracy: 0.9620 - val_loss: 0.0091 - val_accuracy: 0.9623
Epoch 59/100
91074/91074 [==============================] - 1s 9us/sample - loss: 0.0090 - accuracy: 0.9628 - val_loss: 0.0092 - val_accuracy: 0.9613
Epoch 60/100
91074/91074 [==============================] - 1s 6us/sample - loss: 0.0091 - accuracy: 0.9628 - val_loss: 0.0093 - val_accuracy: 0.9600
Epoch 61/100
91074/91074 [==============================] - 1s 6us/sample - loss: 0.0091 - accuracy: 0.9609 - val_loss: 0.0092 - val_accuracy: 0.9604
Epoch 62/100
91074/91074 [==============================] - 1s 10us/sample - loss: 0.0091 - accuracy

2022-09-28 13:56:57.336189: I tensorflow/core/profiler/lib/profiler_session.cc:225] Profiler session started.
2022-09-28 13:56:57.336223: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1307] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:56:57.336229: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1346] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:56:57.339575: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1329] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI could not be loaded or symbol could not be found.
2022-09-28 13:56:57.339597: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:88]  GpuTracer has collect

91074/91074 [==============================] - 1s 10us/sample - loss: 0.1535 - accuracy: 0.1896 - val_loss: 0.0959 - val_accuracy: 0.4720
Epoch 2/100
91074/91074 [==============================] - 1s 8us/sample - loss: 0.0704 - accuracy: 0.5173 - val_loss: 0.0613 - val_accuracy: 0.6122
Epoch 3/100
91074/91074 [==============================] - 1s 8us/sample - loss: 0.0472 - accuracy: 0.7065 - val_loss: 0.0416 - val_accuracy: 0.7546
Epoch 4/100
91074/91074 [==============================] - 1s 8us/sample - loss: 0.0337 - accuracy: 0.8069 - val_loss: 0.0300 - val_accuracy: 0.8217
Epoch 5/100
91074/91074 [==============================] - 1s 6us/sample - loss: 0.0251 - accuracy: 0.8591 - val_loss: 0.0230 - val_accuracy: 0.8613
Epoch 6/100
91074/91074 [==============================] - 1s 9us/sample - loss: 0.0198 - accuracy: 0.8873 - val_loss: 0.0178 - val_accuracy: 0.9116
Epoch 7/100
91074/91074 [==============================] - 1s 11us/sample - loss: 0.0171 - accuracy: 0.9076 - val_los

91074/91074 [==============================] - 1s 12us/sample - loss: 0.0061 - accuracy: 0.9707 - val_loss: 0.0070 - val_accuracy: 0.9619
Epoch 56/100
91074/91074 [==============================] - 1s 7us/sample - loss: 0.0057 - accuracy: 0.9712 - val_loss: 0.0052 - val_accuracy: 0.9710
Epoch 57/100
91074/91074 [==============================] - 1s 8us/sample - loss: 0.0060 - accuracy: 0.9706 - val_loss: 0.0059 - val_accuracy: 0.9720
Epoch 58/100
91074/91074 [==============================] - 1s 11us/sample - loss: 0.0062 - accuracy: 0.9708 - val_loss: 0.0053 - val_accuracy: 0.9751
Epoch 59/100
91074/91074 [==============================] - 1s 10us/sample - loss: 0.0056 - accuracy: 0.9730 - val_loss: 0.0051 - val_accuracy: 0.9762
Epoch 60/100
91074/91074 [==============================] - 1s 12us/sample - loss: 0.0059 - accuracy: 0.9704 - val_loss: 0.0056 - val_accuracy: 0.9743
Epoch 61/100
91074/91074 [==============================] - 1s 14us/sample - loss: 0.0057 - accuracy: 0.9716 

In [44]:
test_list = []

for idx in range(kfold_len):
    autoencoder = load_model(f'model{idx}.h5', custom_objects = {'root_mean_squared_error' : root_mean_squared_error})
    predictions = autoencoder.predict(X_val)
    mse = np.mean(np.power(X_val - predictions, 2), axis=1)
    error_df = pd.DataFrame({'reconstruction_error': mse})
    
    means = error_df.describe()['reconstruction_error'].loc['mean']
    stds = error_df.describe()['reconstruction_error'].loc['std']
        
    median = error_df.describe()['reconstruction_error'].loc['50%']
    IQR = error_df.describe()['reconstruction_error'].loc['75%'] - error_df.describe()['reconstruction_error'].loc['25%']

    threshold = means + 2 * stds
    threshold2 = median + 1.5 * IQR
    
    tests = np.where(mse>=threshold, 1, 0)
    test_list.append(tests)
    

pred = pd.DataFrame(np.array(test_list).T)

5개의 모델에서 추론한 평균값들이 mse임계값이 일정이상인 열이 6개 이상인 경우 비정상 데이터라고 판단

In [45]:
pred['sums'] = pred.sum(axis=1)
pred['sums'].value_counts().sort_index()

0    28169
1      253
2        9
3        7
4        5
5       19
Name: sums, dtype: int64

testset 추론 / 앙상블

In [40]:
test_list = []

for idx in range(kfold_len):
    autoencoder = load_model(f'model{idx}.h5', custom_objects = {'root_mean_squared_error' : root_mean_squared_error})
    predictions = autoencoder.predict(X_test)
    mse = np.mean(np.power(X_test - predictions, 2), axis=1)
    error_df = pd.DataFrame({'reconstruction_error': mse})
    
    means = error_df.describe()['reconstruction_error'].loc['mean']
    stds = error_df.describe()['reconstruction_error'].loc['std']

    median = error_df.describe()['reconstruction_error'].loc['50%']
    IQR = error_df.describe()['reconstruction_error'].loc['75%'] - error_df.describe()['reconstruction_error'].loc['25%']

    threshold = means + 2 * stds
    threshold2 = median + 1.5 * IQR
    
    tests = np.where(mse>=threshold, 1, 0)
    test_list.append(tests)
    
pred = pd.DataFrame(np.array(test_list).T)

In [41]:
pred['sums'] = pred.sum(axis=1)
pred['sums'].value_counts().sort_index()

0    141256
1       902
2        32
3        70
4        74
5       169
Name: sums, dtype: int64

In [48]:
tests = pred['sums'].apply(lambda x : 1 if (x>=3)&(x<5) else 0)
submit = pd.read_csv('ano/sample_submission.csv')
submit['Class'] = tests
submit.to_csv('ano/ensemble_submit.csv', index=False)

단일 모델 추론

In [49]:
autoencoder = load_model(f'model_final.h5')
predictions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - predictions, 2), axis=1)
    
error_df = pd.DataFrame({'reconstruction_error': mse})
error_df.describe()

means = error_df.describe()['reconstruction_error'].loc['mean']
stds = error_df.describe()['reconstruction_error'].loc['std']

median = error_df.describe()['reconstruction_error'].loc['50%']
IQR = error_df.describe()['reconstruction_error'].loc['75%'] - error_df.describe()['reconstruction_error'].loc['25%']

threshold = means + 2 * stds
threshold2 = median + 1.5 * IQR

tests = np.where(mse>=threshold, 1, 0)

submit = pd.read_csv('ano/sample_submission.csv')
submit['Class'] = tests
submit.to_csv('ano/submit.csv', index=False)

In [50]:
pd.DataFrame(tests).value_counts()

0    141337
1      1166
dtype: int64

In [ ]:
submit = pd.read_csv('ano/sample_submission.csv')
submit['Class'] = tests
submit.to_csv('ano/single_submit.csv', index=False)